In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats
import seaborn as sns
import googlemaps
import altair as alt
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('Medicare_2021_colclean.csv',sep='|', low_memory=True)

C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\220912128.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Medicare_2021_colclean.csv',sep='|', low_memory=True)


In [3]:
# Select columns with 'float64' dtype
float64_cols = list(df.select_dtypes(include='float64'))
# Convert those columns to float32
df[float64_cols] = df[float64_cols].astype('float32')

# Select columns with 'float64' dtype
int64_cols = list(df.select_dtypes(include='int64'))
# Convert those columns to float32
df[int64_cols] = df[int64_cols].astype('int32')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9886177 entries, 0 to 9886176
Data columns (total 29 columns):
 #   Column              Dtype  
---  ------              -----  
 0   NPI                 int32  
 1   Last_Org_Name       object 
 2   First_Name          object 
 3   MI                  object 
 4   Crdntls             object 
 5   Gndr                object 
 6   Ent_Cd              object 
 7   St1                 object 
 8   St2                 object 
 9   City                object 
 10  State_Abrvtn        object 
 11  State_FIPS          object 
 12  Zip5                object 
 13  RUCA                float32
 14  RUCA_Desc           object 
 15  Cntry               object 
 16  Type                object 
 17  Mdcr_Prtcptg_Ind    object 
 18  HCPCS_Cd            object 
 19  HCPCS_Desc          object 
 20  HCPCS_Drug_Ind      object 
 21  Place_Of_Srvc       object 
 22  Tot_Benes           int32  
 23  Tot_Srvcs           float32
 24  Tot_Bene_Day_Srvcs  int3

# Benford's Law

In [5]:
df['Avg_Sbmtd_Chrg_str'] = df['Avg_Sbmtd_Chrg'].astype(str)

In [6]:
df['Avg_Sbmtd_Chrg_str'] = df['Avg_Sbmtd_Chrg_str'].str.lstrip('0').str.replace('.', '').str.lstrip('0')

C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\2266731611.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Avg_Sbmtd_Chrg_str'] = df['Avg_Sbmtd_Chrg_str'].str.lstrip('0').str.replace('.', '').str.lstrip('0')


In [7]:
df['first_digit'] = df['Avg_Sbmtd_Chrg_str'].str.get(0)
df['second_digit'] = df['Avg_Sbmtd_Chrg_str'].str.get(1)
df['first_second_digit'] = df['first_digit']+df['second_digit']
df_filter = df[df['Avg_Sbmtd_Chrg']>=10]

In [8]:
def assess_mad(mad):
    """
    Assess the Mean Absolute Deviation (MAD) based on specified thresholds.

    Args:
        mad (float): Mean Absolute Deviation value.

    Returns:
        str: Assessment of conformity.
    """
    if mad <= 0.0012:
        return f"Close conformity (MAD: {mad:.4f})"
    elif mad <= 0.0018:
        return f"Acceptable conformity (MAD: {mad:.4f})"
    elif mad <= 0.0022:
        return f"Marginally acceptable conformity (MAD: {mad:.4f})"
    else:
        return f"Nonconformity (MAD: {mad:.4f})"



## First-Order First Digit Test

In [9]:
observed_first = np.bincount(df.first_digit)[1:]
observed_first = observed_first / len(df.first_digit)
expected_first = np.log10(1 + 1 / np.arange(1, 10))
title = 'First-Order First Digit Test'

In [10]:
data = {
    'digits': np.arange(1, 10),
    'observed_first': observed_first,
    'expected_first': expected_first
}

# Create the DataFrame
df_first = pd.DataFrame(data)

# Display the resulting DataFrame
df_first

,digits,observed_first,expected_first
0,1,0.308235,0.301030
1,2,0.196831,0.176091
2,3,0.132156,0.124939
3,4,0.095420,0.096910
4,5,0.077131,0.079181
5,6,0.060432,0.066947
6,7,0.048637,0.057992
7,8,0.042170,0.051153
8,9,0.038987,0.045757


In [11]:
mad_value = np.mean(np.abs(observed_first - expected_first))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_first).mark_line().encode(
    alt.X("digits:Q").title("Digits"),
    alt.Y("expected_first:Q").title("Probability"),
    alt.Color(datum='expected_first')
    
)

bar = alt.Chart(df_first).mark_bar().encode(
    alt.X("digits:Q").title("Digits"),
    alt.Y("observed_first:Q").title("Probability"),
    alt.Color(datum='observed_first')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)

alt.LayerChart(...)

# First-Order Second Digit Test

In [12]:
observed_second = np.bincount(df_filter.second_digit)
observed_second = observed_second / len(df_filter.second_digit)
expected_second = np.array([np.sum([np.log10(1 + 1/(n+d)) for d in range(10,100,10)]) for n in range(0,10)])
title = 'First-Order Second Digit Test'

In [13]:
title = 'First-Order Second Digit Test'
data = {
    'digits': np.arange(0, 10),
    'observed_second': observed_second,
    'expected_second': expected_second
}

# Create the DataFrame
df_second = pd.DataFrame(data)

# Display the resulting DataFrame
df_second

,digits,observed_second,expected_second
0,0,0.171691,0.119679
1,1,0.097577,0.113890
2,2,0.102162,0.108821
3,3,0.089193,0.104330
4,4,0.088408,0.100308
5,5,0.133773,0.096677
6,6,0.087764,0.093375
7,7,0.080859,0.090352
8,8,0.075631,0.087570
9,9,0.072943,0.084997


In [14]:
mad_value = np.mean(np.abs(observed_second - expected_second))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_second).mark_line().encode(
    alt.X("digits:Q").title("Digits"),
    alt.Y("expected_second:Q").title("Probability"),
    alt.Color(datum='expected_second')
    
)

bar = alt.Chart(df_second).mark_bar().encode(
    alt.X("digits:Q").title("Digits"),
    alt.Y("observed_second:Q").title("Probability"),
    alt.Color(datum='observed_second')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)


alt.LayerChart(...)

# First-Order First-Two Digits Test

In [15]:
observed_first_second = np.bincount(df_filter.first_second_digit)[10:]
observed_first_second = observed_first_second / len(df_filter.first_second_digit)
expected_first_second = np.log10(1 + 1 / np.arange(10, 100))

title = 'First-Order First-Two Digits Test'

In [16]:
data = {
    'digits': np.arange(10, 100),
    'observed_first_second': observed_first_second,
    'expected_first_second': expected_first_second
}

# Create the DataFrame
df_first_second = pd.DataFrame(data)

# Display the resulting DataFrame
df_first_second 

,digits,observed_first_second,expected_first_second
0,10,0.045217,0.041393
1,11,0.032657,0.037789
2,12,0.036179,0.034762
3,13,0.030423,0.032185
4,14,0.030046,0.029963
...,...,...,...
85,95,0.004993,0.004548
86,96,0.003024,0.004501
87,97,0.003007,0.004454
88,98,0.003241,0.004409


In [17]:
mad_value = np.mean(np.abs(observed_first_second - expected_first_second))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_first_second).mark_line().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("expected_first_second:Q").title("Probability"),
    
    alt.Color(datum='expected_first_second')
    
)

bar = alt.Chart(df_first_second).mark_bar().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("observed_first_second:Q").title("Probability"),
    alt.Color(datum='observed_first_second')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)

alt.LayerChart(...)

# Summation test

In [18]:
df_summation = df[['Avg_Sbmtd_Chrg']].sort_values(by='Avg_Sbmtd_Chrg')
df_summation['diff'] = round(((df_summation.Avg_Sbmtd_Chrg.shift(1)-df_summation.Avg_Sbmtd_Chrg)*1000000000),2)
df_summation['diff_abs'] = df_summation['diff'].abs()
df_summation = df_summation[1:]
df_summation_filter = df_summation[df_summation.diff_abs != 0]
df_summation_filter['diff_abs_str'] = df_summation_filter['diff_abs'].astype(str).str.replace('.', '')
df_summation_filter['first_digit'] = df_summation_filter['diff_abs_str'].str.get(0)
df_summation_filter['second_digit'] = df_summation_filter['diff_abs_str'].str.get(1)
df_summation_filter['first_second_digit'] = df_summation_filter['first_digit']+df_summation_filter['second_digit']

C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\1628224554.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_summation_filter['diff_abs_str'] = df_summation_filter['diff_abs'].astype(str).str.replace('.', '')
C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\1628224554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summation_filter['diff_abs_str'] = df_summation_filter['diff_abs'].astype(str).str.replace('.', '')
C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\1628224554.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [19]:
summation = df.groupby('first_second_digit')['Avg_Sbmtd_Chrg'].sum()
summation.drop(['5e'], inplace = True)
proportion = summation / summation.sum()

title = 'Summation Test'

In [20]:
data = {
    'digits': np.arange(10, 100),
    'observed_summation': proportion.values,
    'expected_summation': 0.0111
}

# Create the DataFrame
df_summation = pd.DataFrame(data)

# Display the resulting DataFrame
df_summation

,digits,observed_summation,expected_summation
0,10,0.038338,0.0111
1,11,0.031522,0.0111
2,12,0.032702,0.0111
3,13,0.029840,0.0111
4,14,0.029968,0.0111
...,...,...,...
85,95,0.004591,0.0111
86,96,0.003735,0.0111
87,97,0.003847,0.0111
88,98,0.003693,0.0111


In [21]:
mad_value = np.mean(np.abs(df_summation.observed_summation - df_summation.expected_summation))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_summation).mark_line().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("expected_summation:Q").title("Probability"),
    alt.Color(datum='expected_summation')
    
)

bar = alt.Chart(df_summation).mark_bar().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("observed_summation:Q").title("Probability"),
    alt.Color(datum='observed_summation')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)

alt.LayerChart(...)

# Second Order First-Two Digits Test

In [22]:
obs_first_second = np.bincount(df_summation_filter.first_second_digit)[10:]
obs_first_second = obs_first_second / len(df_summation_filter.first_second_digit)
exp_first_second = np.log10(1 + 1 / np.arange(10, 100))

title = 'Second Order First-Two Digits Test'

In [23]:
data = {
    'digits': np.arange(10, 100),
    'obs_first_second': obs_first_second,
    'exp_first_second': exp_first_second
}

# Create the DataFrame
df_secondorder_first_second = pd.DataFrame(data)

# Display the resulting DataFrame
df_secondorder_first_second 

,digits,obs_first_second,exp_first_second
0,10,0.039479,0.041393
1,11,0.025514,0.037789
2,12,0.044766,0.034762
3,13,0.026858,0.032185
4,14,0.021816,0.029963
...,...,...,...
85,95,0.003945,0.004548
86,96,0.001067,0.004501
87,97,0.007188,0.004454
88,98,0.000684,0.004409


In [24]:
mad_value = np.mean(np.abs(obs_first_second - exp_first_second))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_secondorder_first_second).mark_line().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("exp_first_second:Q").title("Probability"),
    
    alt.Color(datum='exp_first_second')
    
)

bar = alt.Chart(df_secondorder_first_second).mark_bar().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[8, 102])),
    alt.Y("obs_first_second:Q").title("Probability"),
    alt.Color(datum='obs_first_second')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)

alt.LayerChart(...)

# Last two digit

In [25]:
df['lfirst_digit'] = df['Avg_Sbmtd_Chrg_str'].str.get(-2)
df['lsecond_digit'] = df['Avg_Sbmtd_Chrg_str'].str.get(-1)
df['last_two_digit'] = df['lfirst_digit']+df['lsecond_digit']
df_lt = df[df['Avg_Sbmtd_Chrg']>=10]

In [26]:
observed_last_two = np.bincount(df_lt.last_two_digit)
observed_last_two = observed_last_two / len(df_lt.last_two_digit)

title = 'Last-Two Test'

In [27]:
data = {
    'digits': np.arange(0, 100),
    'observed_last_two': observed_last_two,
    'expected_last_two': 0.01
}

# Create the DataFrame
df_last_two = pd.DataFrame(data)

# Display the resulting DataFrame
df_last_two

,digits,observed_last_two,expected_last_two
0,0,0.182823,0.01
1,1,0.002057,0.01
2,2,0.002450,0.01
3,3,0.002666,0.01
4,4,0.003677,0.01
...,...,...,...
95,95,0.005095,0.01
96,96,0.003680,0.01
97,97,0.002644,0.01
98,98,0.002522,0.01


In [29]:
mad_value = np.mean(np.abs(df_last_two.observed_last_two - df_last_two.expected_last_two))
mad_assessment = assess_mad(mad_value)

line = alt.Chart(df_last_two).mark_line().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[-1, 102])),
    alt.Y("expected_last_two:Q").title("Probability"),
    
    alt.Color(datum='expected_last_two')
    
)

bar = alt.Chart(df_last_two).mark_bar().encode(
    alt.X("digits:Q", title="Digits", scale=alt.Scale(domain=[-1, 102])),
    alt.Y("observed_last_two:Q").title("Probability"),
    alt.Color(datum='observed_last_two')
)

bar+line.properties(width=550, height=400, title=f'{title} | {mad_assessment}',)

alt.LayerChart(...)

# Largest growth test

In [30]:
df2 = pd.read_csv('Medicare_2020.csv', encoding='ISO-8859-1')

C:\Users\ronal\AppData\Local\Temp\ipykernel_2216\1904880618.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Medicare_2020.csv', encoding='ISO-8859-1')


In [31]:
df2.head()

,Rndrng_NPI,Rndrng_Prvdr_Last_Org_Name,Rndrng_Prvdr_First_Name,Rndrng_Prvdr_MI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Ent_Cd,Rndrng_Prvdr_St1,Rndrng_Prvdr_St2,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,Rndrng_Prvdr_Cntry,Rndrng_Prvdr_Type,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,HCPCS_Cd,HCPCS_Desc,HCPCS_Drug_Ind,Place_Of_Srvc,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Amt
0,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,24,20817,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99217,Hospital observation care on day of discharge,N,F,23,23.0,23,406.173913,76.803913,61.401739,58.003913
1,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,24,20817,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99218,"Hospital observation care, typically 30 minutes",N,F,16,16.0,16,811.812500,108.249375,85.081250,78.315000
2,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,24,20817,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99220,"Hospital observation care, typically 70 minutes",N,F,16,16.0,16,1150.937500,191.960625,152.376875,146.608750
3,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,24,20817,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99221,"Initial hospital inpatient care, typically 30 ...",N,F,12,12.0,12,681.666667,110.047500,82.237500,76.396667
4,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,MD,24,20817,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99223,"Initial hospital inpatient care, typically 70 ...",N,F,52,52.0,52,1108.384615,209.423846,170.834615,164.659423


In [32]:
df2.rename(columns={"Rndrng_NPI": "NPI"}, inplace=True)

In [34]:
npi_St1 =df.groupby('NPI')['St1'].unique().reset_index()
npi_St1.columns = ['NPI', 'St1']

npi_Zip5 =df.groupby('NPI')['Zip5'].unique().reset_index()
npi_Zip5.columns = ['NPI', 'Zip5']

npi_Ent_cd =df.groupby('NPI')['Ent_Cd'].unique().reset_index()
npi_Ent_cd.columns = ['NPI', 'Ent_cd']

npi_counts = df['NPI'].value_counts().reset_index()
npi_counts.columns = ['NPI', 'Count_2021']

npi_sums = df.groupby('NPI')['Avg_Sbmtd_Chrg'].sum().reset_index()
npi_sums.columns = ['NPI', 'Sum_2021']

npi_mean = df.groupby('NPI')['Avg_Sbmtd_Chrg'].mean().reset_index()
npi_mean.columns = ['NPI', 'Mean_2021']

npi_median = df.groupby('NPI')['Avg_Sbmtd_Chrg'].median().reset_index()
npi_median.columns = ['NPI', 'Median_2021']

npi_tot = df.groupby('NPI')['Tot_Benes'].sum().reset_index()
npi_tot.columns = ['NPI', 'Tot_Benes_2021']

npi_counts1 = df2['NPI'].value_counts().reset_index()
npi_counts1.columns = ['NPI', 'Count_2020']

npi_sums1 = df2.groupby('NPI')['Avg_Sbmtd_Chrg'].sum().reset_index()
npi_sums1.columns = ['NPI', 'Sum_2020']

npi_mean1 = df2.groupby('NPI')['Avg_Sbmtd_Chrg'].mean().reset_index()
npi_mean1.columns = ['NPI', 'Mean_2020']

npi_median1 = df2.groupby('NPI')['Avg_Sbmtd_Chrg'].median().reset_index()
npi_median1.columns = ['NPI', 'Median_2020']

npi_tot1 = df2.groupby('NPI')['Tot_Benes'].sum().reset_index()
npi_tot1.columns = ['NPI', 'Tot_Benes_2020']

merged_df = npi_counts.merge(
    npi_St1, on='NPI', how='outer').merge(
    npi_Ent_cd, on='NPI', how='outer').merge(
    npi_sums, on='NPI', how='outer').merge(
    npi_mean, on='NPI',  how='outer').merge(
    npi_median, on='NPI',  how='outer').merge(
    npi_tot, on='NPI',  how='outer').merge(
    npi_counts1, on='NPI',  how='outer').merge(
    npi_sums1, on='NPI',  how='outer').merge(
    npi_mean1, on='NPI',  how='outer').merge(
    npi_median1, on='NPI',  how='outer').merge(
    npi_tot1, on='NPI',  how='outer')



In [35]:
merged_df_2021 = npi_counts.merge(
    npi_Ent_cd, on='NPI', how='outer').merge(
    npi_sums, on='NPI', how='outer').merge(
    npi_mean, on='NPI',  how='outer').merge(
    npi_median, on='NPI',  how='outer').merge(
    npi_tot, on='NPI',  how='outer')

In [36]:
merged_df_2021.shape

(1123589, 7)

In [37]:
merged_df_2020 = npi_counts1.merge(
    npi_counts1, on='NPI',  how='outer').merge(
    npi_sums1, on='NPI',  how='outer').merge(
    npi_mean1, on='NPI',  how='outer').merge(
    npi_median1, on='NPI',  how='outer').merge(
    npi_tot1, on='NPI',  how='outer')

In [38]:
merged_df_2020.shape

(1085313, 7)

In [39]:
merged = merged_df_2021.merge(merged_df_2020, on='NPI', how='left')

In [40]:
merged.shape

(1123589, 13)

In [41]:
merged.isnull().sum()

NPI                    0
Count_2021             0
Ent_cd                 0
Sum_2021               0
Mean_2021              0
Median_2021            0
Tot_Benes_2021         0
Count_2020_x      127498
Count_2020_y      127498
Sum_2020          127498
Mean_2020         127498
Median_2020       127498
Tot_Benes_2020    127498
dtype: int64

In [42]:
merged_df['Count_growth'] = (merged_df.Count_2021/merged_df.Count_2020)
merged_df['Sum_growth'] = (merged_df.Sum_2021/merged_df.Sum_2020)
merged_df['Tot_bene_growth'] = (merged_df.Tot_Benes_2021/merged_df.Tot_Benes_2020)
merged_df['Sum_bene_ratio'] = (merged_df.Sum_growth/merged_df.Tot_bene_growth)

In [43]:
#Filter check
merged_df[merged_df.Sum_bene_ratio> 3].sort_values(by='Sum_bene_ratio', ascending = False)

,NPI,Count_2021,St1,Ent_cd,Sum_2021,Mean_2021,Median_2021,Tot_Benes_2021,Count_2020,Sum_2020,Mean_2020,Median_2020,Tot_Benes_2020,Count_growth,Sum_growth,Tot_bene_growth,Sum_bene_ratio
1021273,1962830836,1.0,[1200 Callowhill St],[I],190.000000,190.000000,190.000000,11.0,2.0,0.020000,0.010000,0.010000,25.0,0.500000,9500.000000,0.440000,21590.909091
1118961,1356667786,1.0,[Street 111 Km 23.6],[I],64.570000,64.570000,64.570000,17.0,1.0,0.010000,0.010000,0.010000,12.0,1.000000,6456.999969,1.416667,4557.882331
806894,1396716205,3.0,[700 2nd St Ne],[I],1649.969971,549.989990,395.000000,40.0,1.0,0.254229,0.254229,0.254229,11.0,3.000000,6490.091550,3.636364,1784.775176
1005370,1952353187,1.0,[940 Country Club Rd],[O],5172.666504,5172.666504,5172.666504,11.0,1.0,3.786036,3.786036,3.786036,13.0,1.000000,1366.248517,0.846154,1614.657338
917203,1447783626,2.0,[100 Ucla Medical Plz Ste 425],[I],1818.000000,909.000000,909.000000,34.0,1.0,0.520000,0.520000,0.520000,14.0,2.000000,3496.153846,2.428571,1439.592760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785696,1912295437,3.0,[5301 Mcauley Dr],[I],1756.593872,585.531311,527.209290,70.0,1.0,368.000000,368.000000,368.000000,44.0,3.000000,4.773353,1.590909,3.000393
1005526,1508859521,1.0,[1140 Lake St],[I],382.177521,382.177521,382.177521,16.0,2.0,278.646730,139.323365,139.323365,35.0,0.500000,1.371549,0.457143,3.000262
629573,1063429629,5.0,[105 Atsion Rd],[I],1017.633545,203.526718,139.816086,182.0,2.0,205.000000,102.500000,102.500000,110.0,2.500000,4.964066,1.654545,3.000260
257116,1720076938,11.0,[800 Town And Country Blvd Ste 300],[I],7153.114258,650.283081,527.354553,627.0,9.0,2927.984890,325.331654,288.857143,770.0,1.222222,2.443016,0.814286,3.000195


In [44]:
merged_df['ratio_flag'] = np.where(merged_df.Sum_bene_ratio> 2,1,0)

# Relative size factor test

In [45]:
npi_max = df.groupby('NPI')['Avg_Sbmtd_Chrg'].max().reset_index()
npi_max.columns = ['NPI', 'max_1']

In [46]:
df_sorted = df.sort_values('Avg_Sbmtd_Chrg', ascending=False)

df_dropped = df_sorted.drop_duplicates(subset='NPI', keep='first')

npi_max2 = df[~df.isin(df_dropped)].groupby('NPI')['Avg_Sbmtd_Chrg'].max().reset_index()
npi_max2.columns = ['NPI', 'max_2']


In [47]:
merged_df = merged_df.merge(
    npi_max, on='NPI', how='left').merge(
    npi_max2, on='NPI', how='left')

In [48]:
merged_df['rsf'] = merged_df.max_1/merged_df.max_2

In [49]:
#Filter check
merged_df[merged_df.rsf> 2].sort_values(by='rsf', ascending = False)

,NPI,Count_2021,St1,Ent_cd,Sum_2021,Mean_2021,Median_2021,Tot_Benes_2021,Count_2020,Sum_2020,Mean_2020,Median_2020,Tot_Benes_2020,Count_growth,Sum_growth,Tot_bene_growth,Sum_bene_ratio,ratio_flag,max_1,max_2,rsf
967886,1730468273,2.0,[1265 Union Avenue],[I],1350.010010,675.005005,675.005005,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1350.000000,0.010000,135000.000000
925101,1083117766,2.0,[6681 Ridge Rd Ste 300],[I],1244.010010,622.005005,622.005005,206.0,4.0,518.240000,129.560000,122.750000,240.0,0.500000,2.400452,0.858333,2.796643,1,1244.000000,0.010000,124400.000000
949031,1760963201,2.0,[8601 Turnpike Dr Unit 200],[I],850.010010,425.005005,425.005005,22.0,3.0,1086.800000,362.266667,380.600000,60.0,0.666667,0.782122,0.366667,2.133060,1,850.000000,0.010000,85000.000000
969630,1215501507,2.0,[1911 State St],[I],500.010010,250.005005,250.005005,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,500.000000,0.010000,50000.000000
955855,1699256404,2.0,[6918 Gunn Hwy Ste E],[O],82060.546875,41030.273438,41030.273438,40.0,2.0,75001.868152,37500.934076,37500.934076,25.0,1.000000,1.094113,1.600000,0.683821,0,82058.820312,1.729474,47447.277344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670445,1275572026,4.0,[2230 Liliha St],[I],851.950012,212.987503,175.904999,351.0,4.0,1042.350000,260.587500,254.645000,346.0,1.000000,0.817336,1.014451,0.805693,0,362.480011,181.229996,2.000110
455355,1225523590,7.0,[4092 Foxwood Dr Ste 101],[I],4293.170898,613.310120,572.614014,302.0,5.0,2226.000000,445.200000,448.000000,186.0,1.400000,1.928648,1.623656,1.187843,0,1440.000000,719.961548,2.000107
933204,1932701331,2.0,[4000 Cambridge St],[I],860.477295,430.238647,430.238647,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,573.659119,286.818176,2.000079
851784,1477680957,3.0,[960 Joe Frank Harris Pkwy Se],[I],7515.795410,2505.265137,1924.250000,54.0,5.0,12068.312560,2413.662512,2035.941176,110.0,0.600000,0.622771,0.490909,1.268608,0,3848.545410,1924.250000,2.000024


In [50]:
merged_df['rsf_flag'] = np.where(merged_df.rsf> 2,1,0)

In [51]:
merged_df['flag'] = merged_df['rsf_flag']+merged_df['ratio_flag']

# HCPCS flag

In [52]:
HCPCS_Cd_median = df.groupby(['HCPCS_Cd'])['Avg_Sbmtd_Chrg'].median().reset_index()
HCPCS_Cd_median.columns = ['HCPCS_Cd', 'Avg_Sbmtd_Chrg_HCPCS']

In [53]:
df = df.merge(HCPCS_Cd_median, on='HCPCS_Cd', how='left')
df['HCPCS_ratio'] = df.Avg_Sbmtd_Chrg/df.Avg_Sbmtd_Chrg_HCPCS

In [54]:
df['HCPCS_flag'] = np.where(df.HCPCS_ratio> 3,1,0)

In [55]:
npi_hcpcs_flag = df.groupby('NPI')['HCPCS_flag'].sum().reset_index()
npi_hcpcs_flag.columns = ['NPI', 'hcpcs_flag']
merged_df = merged_df.merge(npi_hcpcs_flag, on='NPI', how='left')

In [56]:
#merged_df.to_csv('flag.csv', index=False)

In [57]:
source = pd.DataFrame({
    "category": ['Eligible', 'Not Eligible'],
    "value": [99.16, 0.84]
})

alt.Chart(source).mark_arc(innerRadius=50).encode(
    theta="value",
    color="category:N",
)

alt.Chart(...)

In [58]:
source = pd.DataFrame({
    "category": ['Eligible', 'Not Eligible with Flags'],
    "value": [75.75, 24.25]
})

alt.Chart(source).mark_arc(innerRadius=50).encode(
    theta="value",
    color="category:N",
)

alt.Chart(...)

In [59]:
merged_df.head()

,NPI,Count_2021,St1,Ent_cd,Sum_2021,Mean_2021,Median_2021,Tot_Benes_2021,Count_2020,Sum_2020,Mean_2020,Median_2020,Tot_Benes_2020,Count_growth,Sum_growth,Tot_bene_growth,Sum_bene_ratio,ratio_flag,max_1,max_2,rsf,rsf_flag,flag,hcpcs_flag
0,1538144910,656.0,[1447 York Ct],[O],98401.906250,150.002899,115.084061,10948038.0,643.0,97027.700598,150.898446,114.635437,10342594.0,1.020218,1.014163,1.058539,0.958078,0,1165.704712,1122.043701,1.038912,0,0,30.0
1,1891731626,640.0,[4225 E Fowler Ave],[O],108890.328125,170.141144,122.531143,5586675.0,624.0,110824.887552,177.603986,124.422291,5467219.0,1.025641,0.982544,1.021849,0.961535,0,2762.000000,2504.675049,1.102738,0,0,36.0
2,1932145778,622.0,[1 Insights Dr],[O],105890.328125,170.241684,126.134476,5296381.0,600.0,97742.967041,162.904945,128.434379,4111244.0,1.036667,1.083355,1.288267,0.840940,0,3082.260254,2762.000000,1.115952,0,0,30.0
3,1063497451,613.0,[69 1st Ave],[O],98077.500000,159.995926,117.949997,6877772.0,591.0,95401.872824,161.424489,117.948137,6108559.0,1.037225,1.028046,1.125924,0.913069,0,1160.750854,1122.000000,1.034537,0,0,31.0
4,1366479099,603.0,[8401 Fallbrook Ave],[O],100922.781250,167.367798,120.351227,4854261.0,592.0,96616.018640,163.202734,118.221494,4414351.0,1.018581,1.044576,1.099655,0.949913,0,3050.000000,2990.000000,1.020067,0,0,32.0


In [60]:
merged_df.shape

(1212811, 24)

In [61]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212811 entries, 0 to 1212810
Data columns (total 24 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   NPI              1212811 non-null  int64  
 1   Count_2021       1123589 non-null  float64
 2   St1              1123589 non-null  object 
 3   Ent_cd           1123589 non-null  object 
 4   Sum_2021         1123589 non-null  float32
 5   Mean_2021        1123589 non-null  float32
 6   Median_2021      1123589 non-null  float32
 7   Tot_Benes_2021   1123589 non-null  float64
 8   Count_2020       1085313 non-null  float64
 9   Sum_2020         1085313 non-null  float64
 10  Mean_2020        1085313 non-null  float64
 11  Median_2020      1085313 non-null  float64
 12  Tot_Benes_2020   1085313 non-null  float64
 13  Count_growth     996091 non-null   float64
 14  Sum_growth       996091 non-null   float64
 15  Tot_bene_growth  996091 non-null   float64
 16  Sum_bene_ratio   9

In [ ]:
#df_first.to_csv('df_first.csv', index=False)
#df_second.to_csv('df_second.csv', index=False)
#df_first_second.to_csv('df_first_second.csv', index=False)
#df_summation.to_csv('df_summation.csv', index=False)
#df_secondorder_first_second.to_csv('df_secondorder_first_second.csv', index=False)
#df_last_two.to_csv('df_last_two.csv', index=False)